In [1]:
import numpy as np
from numpy import genfromtxt

In [2]:
data = genfromtxt('PhotosLARC/bank_note_data.txt', delimiter=',')

In [3]:
# Labels at last col (0 = forgery, 1 = valid dollar)
data

array([[  3.6216 ,   8.6661 ,  -2.8073 ,  -0.44699,   0.     ],
       [  4.5459 ,   8.1674 ,  -2.4586 ,  -1.4621 ,   0.     ],
       [  3.866  ,  -2.6383 ,   1.9242 ,   0.10645,   0.     ],
       ...,
       [ -3.7503 , -13.4586 ,  17.5932 ,  -2.7771 ,   1.     ],
       [ -3.5637 ,  -8.3827 ,  12.393  ,  -1.2823 ,   1.     ],
       [ -2.5419 ,  -0.65804,   2.6842 ,   1.1952 ,   1.     ]])

In [4]:
labels = data[:,4]

In [5]:
labels

array([0., 0., 0., ..., 1., 1., 1.])

In [6]:
features = data[:,0:4]

In [7]:
features

array([[  3.6216 ,   8.6661 ,  -2.8073 ,  -0.44699],
       [  4.5459 ,   8.1674 ,  -2.4586 ,  -1.4621 ],
       [  3.866  ,  -2.6383 ,   1.9242 ,   0.10645],
       ...,
       [ -3.7503 , -13.4586 ,  17.5932 ,  -2.7771 ],
       [ -3.5637 ,  -8.3827 ,  12.393  ,  -1.2823 ],
       [ -2.5419 ,  -0.65804,   2.6842 ,   1.1952 ]])

In [8]:
X = features
y = labels

In [9]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split

In [10]:
# 33% of data will be part of the test set (random state is rand seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
# Scaling data (helps in case that magnitude from min and max is huge, in this case is just called by convention)
from sklearn.preprocessing import MinMaxScaler

In [14]:
scaler_object = MinMaxScaler()

In [15]:
scaler_object.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [16]:
scaled_X_train = scaler_object.transform(X_train)

In [18]:
scaled_X_test = scaler_object.transform(X_test)

In [20]:
# Creating model with Keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [21]:
model = Sequential()
# Creates a model with 4 neurons, that expects 4 features (the 4 cols of data from txt file), using activation funciton ReLu
model.add(Dense(4, input_dim=4,activation='relu'))

# No need for input dim specification because this is a hidden layer
model.add(Dense(8, activation='relu'))

# Only 1 neuron because it's output layer (sigmoid is fitted from 0 and 1, which is what we want (true or false))
model.add(Dense(1,activation='sigmoid'))

In [22]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [23]:
# Training the model (epcho is the #iterations of going through the data, verbose is for printing state while training model)
model.fit(scaled_X_train, y_train, epochs=50,verbose=2)

Epoch 1/50
 - 1s - loss: 0.6970 - acc: 0.4516
Epoch 2/50
 - 0s - loss: 0.6855 - acc: 0.4810
Epoch 3/50
 - 0s - loss: 0.6788 - acc: 0.5397
Epoch 4/50
 - 0s - loss: 0.6738 - acc: 0.6681
Epoch 5/50
 - 0s - loss: 0.6692 - acc: 0.7225
Epoch 6/50
 - 0s - loss: 0.6642 - acc: 0.7280
Epoch 7/50
 - 0s - loss: 0.6570 - acc: 0.7519
Epoch 8/50
 - 0s - loss: 0.6498 - acc: 0.7573
Epoch 9/50
 - 0s - loss: 0.6372 - acc: 0.7802
Epoch 10/50
 - 0s - loss: 0.6163 - acc: 0.7954
Epoch 11/50
 - 0s - loss: 0.5952 - acc: 0.7976
Epoch 12/50
 - 0s - loss: 0.5744 - acc: 0.8118
Epoch 13/50
 - 0s - loss: 0.5534 - acc: 0.8139
Epoch 14/50
 - 0s - loss: 0.5334 - acc: 0.8313
Epoch 15/50
 - 0s - loss: 0.5139 - acc: 0.8281
Epoch 16/50
 - 0s - loss: 0.4953 - acc: 0.8422
Epoch 17/50
 - 0s - loss: 0.4775 - acc: 0.8466
Epoch 18/50
 - 0s - loss: 0.4616 - acc: 0.8498
Epoch 19/50
 - 0s - loss: 0.4464 - acc: 0.8596
Epoch 20/50
 - 0s - loss: 0.4328 - acc: 0.8683
Epoch 21/50
 - 0s - loss: 0.4192 - acc: 0.8749
Epoch 22/50
 - 0s - lo

In [25]:
# Using test data for predictions

In [26]:
model.metrics_names

['loss', 'acc']

In [27]:
from sklearn.metrics import confusion_matrix,classification_report

In [28]:
predictions = model.predict_classes(scaled_X_test)

In [30]:
# TP FN
# FP TN
confusion_matrix(y_test,predictions)

array([[250,   7],
       [ 17, 179]])

In [31]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

        0.0       0.94      0.97      0.95       257
        1.0       0.96      0.91      0.94       196

avg / total       0.95      0.95      0.95       453



In [32]:
# Saving and loading model
model.save('forgeryDetector.h5')

In [33]:
from keras.models import load_model

In [34]:
new_model = load_model('forgeryDetector.h5')

In [35]:
new_model.predict_classes(scaled_X_test)

array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
    